In [1]:
import pandas as pd
import numpy as np
# from azure.ai.ml import MLClient
# from azure.identity import DefaultAzureCredential

# ml_client = MLClient.from_config(credential=DefaultAzureCredential())
# data_asset = ml_client.data.get("allo_cine", version="5")

df = pd.read_parquet('../allocine_spider.parquet')
df.head()

,actors,critics_score,date,director,editor,french_boxoffice,french_first_week_boxoffice,french_visa,genre,langage,length,nationality,title,url,viewers_score,vo_title
0,Michelle Pfeiffer|Lucas Hedges|Valerie Mahaffey,2.1,2021-06-20,Azazel Jacobs,Sony Pictures Releasing France,BUG,NaN,-,Comédie|Drame,Anglais,110,Canada|Irlande|France,French Exit,/article/fichearticle_gen_carticle=18706328.html,--,None
1,Margot Nuccetelli|Dave Willetts|Rick Kavanian,2.4,2022-07-20,Adam Gunn,Alba Films,98321,42006.0,157299,Animation|Famille|Fantastique,Anglais,82,Allemagne|Australie|Belgique|Inde,"Mia et moi, L’Héroïne de Centopia",/article/fichearticle_gen_carticle=18711883.html,2.5,Mia And Me - The Movie
2,Éric Nantchouang|Salif Cissé|Édouard Sulpice,3.9,2021-07-21,Guillaume Brac,Arte,11534,5612.0,149445,Comédie,Français,95,France,À l’abordage,/article/fichearticle_gen_carticle=18706984.html,3.7,None
3,Anthony Hopkins|Olivia Colman|Mark Gatiss,3.9,2021-05-26,Florian Zeller,Orange Studio Distribution / UGC Distribution,586834,153927.0,151195,Drame,Anglais,98,Grande-Bretagne|France,The Father,/article/fichearticle_gen_carticle=1000021987....,4.3,None
4,Josiane Balasko|Didier Bourdon|Marilou Berry,2.8,2021-12-15,Alexandra Leclère,UGC Distribution,642557,198247.0,-,Comédie,Français,95,France,Mes très chers enfants,/article/fichearticle_gen_carticle=18708877.html,2.6,None


In [2]:
df.dtypes

actors                                 object
critics_score                          object
date                           datetime64[ns]
director                               object
editor                                 object
french_boxoffice                       object
french_first_week_boxoffice           float64
french_visa                            object
genre                                  object
langage                                object
length                                  int64
nationality                            object
title                                  object
url                                    object
viewers_score                          object
vo_title                               object
dtype: object

In [3]:
df['viewers_score']= pd.to_numeric(df['viewers_score'], errors='coerce')
df['critics_score']= pd.to_numeric(df['critics_score'], errors='coerce')
df['french_boxoffice']= pd.to_numeric(df['french_boxoffice'], errors='coerce')

In [4]:
df['length']= pd.to_numeric(df['length'], errors='coerce')
df['length'] = df['length'].apply(lambda x : np.nan if x == 0 else x )


In [5]:
df['date']= pd.to_datetime(df['date'])

In [6]:
df['genre'] = df['genre'].str.split('|')
df['actors'] = df['actors'].str.split('|')
df['nationality'] = df['nationality'].str.split('|')
df['langage'] = df['langage'].str.split('|') 	

In [7]:
feature_of_interest = [
    'actors',
    'critics_score',
    'date',
    'director',
    'editor',
    'genre',
    'langage',
    'length',
    'nationality',
    'viewers_score'
]

info_film = ['french_visa', 'title', 'vo_title', 'url']


target = 'french_first_week_boxoffice'
data = feature_of_interest
data.append(target)

data.extend(info_film)
df_to_train = df[data]

In [8]:
df_to_train

,actors,critics_score,date,director,editor,genre,langage,length,nationality,viewers_score,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[Michelle Pfeiffer, Lucas Hedges, Valerie Maha...",2.1,2021-06-20,Azazel Jacobs,Sony Pictures Releasing France,"[Comédie, Drame]",[Anglais],110.0,"[Canada, Irlande, France]",NaN,NaN,-,French Exit,None,/article/fichearticle_gen_carticle=18706328.html
1,"[Margot Nuccetelli, Dave Willetts, Rick Kavanian]",2.4,2022-07-20,Adam Gunn,Alba Films,"[Animation, Famille, Fantastique]",[Anglais],82.0,"[Allemagne, Australie, Belgique, Inde]",2.5,42006.0,157299,"Mia et moi, L’Héroïne de Centopia",Mia And Me - The Movie,/article/fichearticle_gen_carticle=18711883.html
2,"[Éric Nantchouang, Salif Cissé, Édouard Sulpice]",3.9,2021-07-21,Guillaume Brac,Arte,[Comédie],[Français],95.0,[France],3.7,5612.0,149445,À l’abordage,None,/article/fichearticle_gen_carticle=18706984.html
3,"[Anthony Hopkins, Olivia Colman, Mark Gatiss]",3.9,2021-05-26,Florian Zeller,Orange Studio Distribution / UGC Distribution,[Drame],[Anglais],98.0,"[Grande-Bretagne, France]",4.3,153927.0,151195,The Father,None,/article/fichearticle_gen_carticle=1000021987....
4,"[Josiane Balasko, Didier Bourdon, Marilou Berry]",2.8,2021-12-15,Alexandra Leclère,UGC Distribution,[Comédie],[Français],95.0,[France],2.6,198247.0,-,Mes très chers enfants,None,/article/fichearticle_gen_carticle=18708877.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248,"[Pio Marmaï, Roschdy Zem, Vincent Lindon]",4.1,2022-02-09,Thierry de Peretti,Pyramide Distribution,"[Biopic, Drame, Thriller]",[Français],123.0,[France],2.7,101542.0,-,Enquête sur un scandale d'état,None,/article/fichearticle_gen_carticle=1000059971....
2249,"[Josephine Langford, Hero Fiennes Tiffin, Dyla...",2.2,2020-12-22,Roger Kumble,-,"[Drame, Erotique, Romance]",[Anglais],106.0,[U.S.A.],NaN,NaN,-,After - Chapitre 2,After We Collided,/film/fichefilm-274369/dvd-blu-ray/?cproduct=2...
2250,"[Nathalie Baye, Lyna Khoudri, Pascale Arbillot]",2.9,2021-11-10,Sylvie Ohayon,UGC Distribution,"[Comédie, Comédie dramatique, Drame]",[Français],101.0,[France],2.9,92692.0,151990,Haute couture,None,/article/fichearticle_gen_carticle=18707786.html
2251,"[Ehsan Mirhosseini, Shaghayegh Shourian, Kaveh...",4.1,2021-12-01,Mohammad Rasoulof,Pyramide Distribution,[Drame],[Persan],152.0,"[Allemagne, République, tchèque, Iran]",4.1,26645.0,153794,Le Diable n'existe pas,Sheytan vojud nadarad,/article/fichearticle_gen_carticle=1000080192....


In [9]:
df.describe()

,critics_score,date,french_boxoffice,french_first_week_boxoffice,length,viewers_score
count,2242.000000,2253,2.086000e+03,2.083000e+03,2248.000000,1952.000000
mean,3.165522,2023-03-19 17:25:00.399467520,2.827935e+05,9.577645e+04,104.265569,3.124744
min,1.000000,2019-11-20 00:00:00,0.000000e+00,0.000000e+00,31.000000,1.100000
25%,2.900000,2022-04-06 00:00:00,8.163000e+03,4.028000e+03,91.000000,2.700000
50%,3.200000,2023-04-05 00:00:00,5.132950e+04,2.128700e+04,101.000000,3.200000
75%,3.500000,2024-03-29 00:00:00,2.122820e+05,8.704350e+04,115.000000,3.600000
max,4.900000,2025-05-28 00:00:00,1.400054e+07,2.867515e+06,214.000000,4.500000
std,0.556045,NaN,7.924740e+05,2.237296e+05,23.544260,0.619475


In [10]:
# from azure.ai.ml.entities import Data
# from azure.ai.ml.constants import AssetTypes
import pyarrow as pa
import pyarrow.parquet as pq

table = pa.Table.from_pandas(df_to_train)
pq.write_table(table, 'allocine_spider_clean.parquet')
# new_data_asset = Data(
#     path='allocine_spider.parquet',
#     type=AssetTypes.URI_FILE,
#     name="allo_cine", 
#     version="1.1",              # nouvelle version
#     description="Premier jet pour entrainer les modèles"
# )

# ml_client.data.create_or_update(new_data_asset)